In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re, string, unicodedata
import os
import inflect
import numpy as np
import pandas as pd
import collections
import networkx as nx
import time
import pickle

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk import download
from nltk.corpus import stopwords


from sklearn.metrics.pairwise import cosine_similarity


%pip install stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=20e48ff68dab2613a29d6b4232b27541ec51b125746921b197befe5caf2a789b
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| depparse     | ancora   |
| sentiment    | tass2020 |
| constituency | combined |
| ner          | conll02  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
#Convierto cada tweet del dataset en un vector de palabras
from nltk import word_tokenize
def ConvertirDataAnterior(corpusBase):
  textos = []
  for y in [corpusBase[i][0] for i in range(len(corpusBase))]:
      doc = word_tokenize(y)
      textos.append(doc)
  return textos

def ConvertirData(data):
  comments = []
  for line in data:
      comments.append(str(line).split(' '))
  return comments

In [ ]:
DataFinal = pd.read_excel('/content/gdrive/My Drive/ART_EMERGENCY/GLOVE_CODIGO/4DataFinalLemaFiltro.xlsx')

FUNCIONES

In [ ]:
def Knn(Matriz,tipo, k):
      labels = tipo
      pred_label = []
      etiquetar = []
      indice = 0
      for x in labels:
          if(indice != len(labels)-1):
            if(Matriz[indice]<=0.25): #Filtro que asigan directament 0 cuando el valor de comparacion sea cero o menor a cero
              etiquetar.append((Matriz[indice], 0))
            else:
              etiquetar.append((Matriz[indice], x))
          indice += 1
      etiquetar.sort(reverse=True)
      neighbors = etiquetar[:k]
      votes = []
      for neighbor in neighbors:
                votes.append(neighbor[1])
      counter = collections.Counter(votes) #Determina el que mas se repite
      pred_label.append(counter.most_common()[0][0])
      return pred_label[0]

MODELO GLOVE

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import transformers
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [ ]:
    import numpy as np
    from sklearn.model_selection import train_test_split
    import pandas as pd
    from sklearn.metrics import accuracy_score


      

    def modeloBert(tweet, train):
      Tweet = tweet #Se Preprocesa el tweet y se tokeniza el texto
      #Tweet = tweet
      if len(Tweet.split()) <= 2: #Se valida que el tweet preprocesado tenga un minino de palabras
        Tweet = 'falso'  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
      
      #print('TWEET I ', Tweet)
      test_inputs = tokenizer.batch_encode_plus([Tweet], max_length=512, truncation=True, pad_to_max_length=True, return_attention_mask=True, return_tensors="pt")

        # Crea los conjuntos de datos de entrenamiento y prueba
      test_dataset = TensorDataset(test_inputs["input_ids"], test_inputs["attention_mask"])

        # Crea los data loaders
      test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset))

      sim = []
      text = []
      label = []
      valores = train

      for i in valores:
        test_inputs2 = tokenizer.batch_encode_plus([i[0]], max_length=512, truncation=True, pad_to_max_length=True, return_attention_mask=True, return_tensors="pt")
        test_dataset2 = TensorDataset(test_inputs2["input_ids"], test_inputs2["attention_mask"])
        test_dataloader2 = DataLoader(test_dataset2, sampler=SequentialSampler(test_dataset2))
        
        for batch in test_dataloader:
          val_tweet = batch[0].numpy()
        
        for line in test_dataloader2:
          val = line[0].numpy()
          similarity = (val[0] @ val_tweet[0]) / (np.linalg.norm (val[0]) * np.linalg.norm (val_tweet[0])) 
          sim.append(similarity)


      return sim


    def predict_Word2Vec(Train, TrainOri, y_train, tweets):
      prediccion = []
      
      for tweet in [tweets[i][0] for i in range(len(tweets))]:
        Similaridad= modeloBert(tweet, Train)
        prediccion.append(k_predict_Word2Vec(Similaridad,y_train))
      return prediccion

    def k_predict_Word2Vec(Similaridad, y_train):
      k_1 = Knn(Similaridad,y_train, 1)
      k_3 = Knn(Similaridad,y_train, 3)
      k_5 = Knn(Similaridad,y_train, 5)
      k_7 = Knn(Similaridad,y_train, 7)
      k_9 = Knn(Similaridad,y_train, 9)
      k_11 = Knn(Similaridad,y_train, 11)
      return k_1, k_3, k_5, k_7, k_9, k_11 


In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = DataFinal['text']
y = DataFinal['tipo']

kf = KFold(n_splits=8, shuffle=True, random_state=2) 

In [ ]:
#MODELO Word2vec
train_data=DataFinal

# Tokeniza la data y convierte a ids
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
train_inputs = tokenizer.batch_encode_plus(train_data["text"].tolist(), max_length=512, truncation=True, pad_to_max_length=True, return_attention_mask=True, return_tensors="pt")
train_labels = torch.tensor(train_data["tipo"].tolist())


# Crea los conjuntos de datos de entrenamiento y prueba
train_dataset = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)

# Crea los data loaders
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset)) 

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

from sklearn.metrics import confusion_matrix
nombre = []
accuracy = []
precision = []
recall = []
sensitivy = []
f1 = []
log = []

indice_fold=1


for entrenamiento_indice, prueba_indice in kf.split(X):
    #print(entrenamiento_indice,prueba_indice)
    X_train, X_test = X[entrenamiento_indice],X[prueba_indice]
    y_train, y_test = y[entrenamiento_indice],y[prueba_indice]
    X_train = X_train.values.reshape([X_train.values.shape[0],1])
    X_test = X_test.values.reshape([X_test.values.shape[0],1]) 
    
    y_pred =  predict_Word2Vec(X_train,X[entrenamiento_indice],y_train,X_test)
    y_pred = np.array(y_pred)
    a = [1,3,5,7,9,11]
    print(' COMPLETADO:')
    for i in range(6):
      name = 'BERT K'+str(a[i])+', FOLD '+str(indice_fold)
      nombre.append(name)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred[:,i]).ravel()
      accuracy.append(metrics.accuracy_score(y_test,y_pred[:,i]))
      precision.append(metrics.precision_score(y_test,y_pred[:,i]))
      f1.append(metrics.f1_score(y_test,y_pred[:,i]))
      log.append(metrics.log_loss(y_test,y_pred[:,i]))
      recall.append( tn / float(tn + fp))
      sensitivy.append(tp / float(tp + fn))
      print(' '+name)
      #print(accuracy, precision, recall, sensitivy)
    indice_fold = indice_fold + 1 

 COMPLETADO:
 BERT K1, FOLD 1
 BERT K3, FOLD 1
 BERT K5, FOLD 1
 BERT K7, FOLD 1
 BERT K9, FOLD 1
 BERT K11, FOLD 1


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


 COMPLETADO:
 BERT K1, FOLD 2
 BERT K3, FOLD 2
 BERT K5, FOLD 2
 BERT K7, FOLD 2
 BERT K9, FOLD 2
 BERT K11, FOLD 2


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


 COMPLETADO:
 BERT K1, FOLD 3
 BERT K3, FOLD 3
 BERT K5, FOLD 3
 BERT K7, FOLD 3
 BERT K9, FOLD 3
 BERT K11, FOLD 3


...

 COMPLETADO:
 BERT K1, FOLD 6
 BERT K3, FOLD 6
 BERT K5, FOLD 6
 BERT K7, FOLD 6
 BERT K9, FOLD 6
 BERT K11, FOLD 6


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


 COMPLETADO:
 BERT K1, FOLD 7
 BERT K3, FOLD 7
 BERT K5, FOLD 7
 BERT K7, FOLD 7
 BERT K9, FOLD 7
 BERT K11, FOLD 7


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


 COMPLETADO:
 BERT K1, FOLD 8
 BERT K3, FOLD 8
 BERT K5, FOLD 8
 BERT K7, FOLD 8
 BERT K9, FOLD 8
 BERT K11, FOLD 8


In [ ]:
resultado2=[nombre,accuracy,precision,recall,sensitivy,f1,log]
Final2 = pd.DataFrame(resultado2)
Final2.T

,0,1,2,3,4,5,6
0,"BERT K1, FOLD 1",0.523952,0.466292,0.491979,0.564626,0.510769,16.44234
1,"BERT K3, FOLD 1",0.553892,0.494318,0.524064,0.591837,0.5387,15.40823
2,"BERT K5, FOLD 1",0.565868,0.505682,0.534759,0.605442,0.551084,14.994587
3,"BERT K7, FOLD 1",0.571856,0.511494,0.545455,0.605442,0.554517,14.787763
4,"BERT K9, FOLD 1",0.571856,0.511494,0.545455,0.605442,0.554517,14.787763
5,"BERT K11, FOLD 1",0.577844,0.517442,0.55615,0.605442,0.557994,14.58094
6,"BERT K1, FOLD 2",0.565868,0.530864,0.575419,0.554839,0.542587,14.994561
7,"BERT K3, FOLD 2",0.580838,0.545455,0.581006,0.580645,0.5625,14.477511
8,"BERT K5, FOLD 2",0.583832,0.548193,0.581006,0.587097,0.566978,14.374101
9,"BERT K7, FOLD 2",0.595808,0.559524,0.586592,0.606452,0.582043,13.960461


In [ ]:
Final2.T.to_excel('Pruebas_BERT.xlsx', index=False)
!cp Pruebas_BERT.xlsx '/content/gdrive/My Drive/ART_EMERGENCY/PRUEBAS/'